In [2]:
#celija za importanje libova
import pandas as pd
import numpy as np
import datetime as dt
import time
import sys

In [2]:
#celija za klasu Dataset
#dodajte novi kod u postojece ili nove funkcije, drzite kod cistim
#ne mojte sarat po celijama jer je nepregledno uzasno
#celija ispod ove ima lijep primjer kako se radi objekt
#funkcija __init__ je konstruktor, zove se kod kreiranja klase i u njoj ucitavamo dani dataset i ubacujemo dodatne makroekonomske
#pokazatelje dok su datumi u originalnoj formi, kasnije radimo sa numeric datumima, tako da dodavajte nove funkcije koje ce se
#pozivat na datasetu umjeto dopunjavanja __init__
#svaku funkciju opisite sto se radi i sto je u kojoj varijabli ako su imena nerazumljiva


#ako neku funkciju iz klase zelite pozvati iz druge funkcije iz klase, morate ju zvat sa self.ime_Fje() a ime_fje() mora u definiciji prvi arugment primit self

class Dataset:
    #data je aktualni dataset
    
    #------------------------------------------------------------------------------------------------------------------ 
    #------------------------------------------------------------------------------------------------------------------
    #konstruktor klase, ucitava podatke iz csv-a sa minimalnom obradom, daljnja obrada se radi u drugim funkcijama
    def __init__(self, extend_range = False):
        #mozgalo dataset
        self.data = pd.read_csv("D:/Mozgalo2019/training_set_economy_features.csv", sep = ',', low_memory=False, error_bad_lines = False) 
    
        #dodatni dataset za BDP i sl
        self.additional = pd.read_csv("API_HRV_DS2_en_csv_v2_10527771.csv", header = 2) #dataset sa worldbank.org
        self.additional = self.additional.drop(axis = 1, labels = ["Country Name", "Country Code", "Indicator Code"])
        self.additional = self.additional.drop(axis = 1, labels = self.additional.columns[[x for x in range (1,37)]])#makni godine od 1960-19999
        self.newAdditional = pd.read_csv("DodatniDataset.csv")
        self.datum_pocetka_krize = MDate("15.09.2008").numericDate
        self.datum_ulaska_u_eu = MDate("01.07.2013").numericDate
        self.datum_pada_agrokora = MDate("02.01.2017").numericDate
        self.extend_range = extend_range
    #------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------------------------------------------------------------------ 
    
    
    #------------------------------------------------------------------------------------------------------------------ 
    #------------------------------------------------------------------------------------------------------------------
    #funkcija koja ucitava dodatni dataset iz svijetske banke
    #GDPPC - gdp per capita
    #GDPRast - rast bdp-a
    def LoadAdditionalData(self):
        self.DataArray = self.LoadDataFrom(self.additional)
        self.GDPRast = self.DataArray[0], 
        self.TaxOnIncome, self.NonFinacialInvestement = self.DataArray[1], self.DataArray[2]
        self.GDPomjer, self.GDP, self.ExportGoods = self.DataArray[3], self.DataArray[4], self.DataArray[5]
        self.TaxesDorian, self.ImportOfGoods, self.Inflation =self.DataArray[6], self.DataArray[7], self.DataArray[8]
        self.ForeignInvest, self.GrossDomesticSaving, self.GrossSaving =self.DataArray[9], self.DataArray[10], self.DataArray[11]
        self.GDPPC, self.Turism, self.TariffLines =self.DataArray[12], self.DataArray[13], self.DataArray[14]
        self.TaxRevenue, self.TaxOnGoods = self.DataArray[15], self.DataArray[16]
    #------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------------------------------------------------------------------ 
    
    
    #------------------------------------------------------------------------------------------------------------------ 
    #------------------------------------------------------------------------------------------------------------------
    #funkcija koja dodaje ucitane podatke svjetske banke u podatke
    def AddAdditionalData(self, table):
        self.AddBDPTimeSeriesToTable(table)
    #------------------------------------------------------------------------------------------------------------------ 
    #------------------------------------------------------------------------------------------------------------------ 
    
    
    #------------------------------------------------------------------------------------------------------------------ 
    #------------------------------------------------------------------------------------------------------------------
    #funkcija koja izvlaci iz dodatnog dataseta potrebne vrijednosti
    def LoadDataFrom(self,additional):                          
        GDPRast = additional.loc[additional["Indicator Name"] == "GDP per capita growth (annual %)"]
        GDPRast = GDPRast.drop(axis = 1, labels = GDPRast.columns[[0]])
                                    
        TaxOnIncome = additional.loc[additional["Indicator Name"] == "Taxes on income, profits and capital gains (% of total taxes)"]
        TaxOnIncome = TaxOnIncome.drop(axis = 1, labels = TaxOnIncome.columns[[0]])
                                    
        NonFinacialInvestement = additional.loc[additional["Indicator Name"] == "Net investment in nonfinancial assets (% of GDP)"] #AN
        NonFinacialInvestement = NonFinacialInvestement.drop(axis = 1, labels = NonFinacialInvestement.columns[[0]])
        
        #new new dataset
        
        GDPomjer = self.newAdditional.loc[self.newAdditional["Series.Name"] == "gdp omjer"]
        GDP =  self.newAdditional.loc[self.newAdditional["Series.Name"] == "GDP (current US$)"]
        ExportGoods =  self.newAdditional.loc[self.newAdditional["Series.Name"] == "Exports of goods and services (current LCU)"]
        TaxesDorian = self.newAdditional.loc[self.newAdditional["Series.Name"] == "Taxes less subsidies on products (current LCU)"]
        ImportOfGoods =  self.newAdditional.loc[self.newAdditional["Series.Name"] == "Imports of goods and services (current LCU)"]
        Inflation =  self.newAdditional.loc[self.newAdditional["Series.Name"] == "Inflation, GDP deflator (annual %)"]
        ForeignInvest =  self.newAdditional.loc[self.newAdditional["Series.Name"] == "Foreign direct investment, net outflows (BoP, current US$)"]
        GrossDomesticSaving = self.newAdditional.loc[self.newAdditional["Series.Name"] == "GDP per capita, PPP (constant 2011 international $)"]
        GrossSaving =  self.newAdditional.loc[self.newAdditional["Series.Name"] == "Gross savings (% of GDP)"]
        GDPPC =  self.newAdditional.loc[self.newAdditional["Series.Name"] == "GDP per capita, PPP (constant 2011 international $)"]
        Turism =  self.newAdditional.loc[self.newAdditional["Series.Name"] == "International tourism, number of arrivals"]
        TariffLines =  self.newAdditional.loc[self.newAdditional["Series.Name"] == "Share of tariff lines with international peaks, all products (%)"]
        TaxRevenue =  self.newAdditional.loc[self.newAdditional["Series.Name"] == "Tax revenue (% of GDP)"]
        TaxOnGoods =  self.newAdditional.loc[self.newAdditional["Series.Name"] == "Taxes on goods and services (% of revenue)"]
        
        return1 = [GDPRast,TaxOnIncome, NonFinacialInvestement, GDPomjer, GDP, ExportGoods, TaxesDorian, ImportOfGoods]
        return2 = [Inflation, ForeignInvest, GrossDomesticSaving, GrossSaving, GDPPC, Turism, TariffLines, TaxRevenue, TaxOnGoods]
        return return1+return2
    #------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------------------------------------------------------------------ 
    
    
    #------------------------------------------------------------------------------------------------------------------ 
    #------------------------------------------------------------------------------------------------------------------
    #dodaje izolirani stupac kao seriju u tablicu po sljedecim pravilima
    #prvi dodani stupac je vrijednost serije u godini izvjestavanja
    #drugi stupac je vrijednost serije u godini datuma otvaranja
    #treci je vrijednost serije u datumu planiranog zatvaranja

    #series je array time serija koji se dodaje
    def AddBDPTimeSeriesToTable(self, table):
        series = self.DataArray
        print("Initializing new features...")
        table["RAST_BDPA_U_GODINI_OTVARANJA"] = float("NaN")
        table["RAST_BDPA_U_GODINI_ZATVARANJA"] = float("NaN")
        table["MAX_RAST_BDP"] = float("NaN")
        table["MIN_RAST_BDP"] = float("NaN")
        table["OTV-KRIZA"] = float("NaN")
        table["ZATV-KRIZA"] = float("NaN")
        table["OTV-EU"] = float("NaN")
        table["ZATV-EU"] = float("NaN")
        table["OTV_TECAJ"] = float("NaN")
        table["ZATV_TECAJ"] = float("NaN")
        table["MAX_TECAJ"] = float("NaN")
        table["MIN_TECAJ"] = float("NaN")
        table["POREZ_NA_DOBIT_OTV"] = float("NaN")
        table["POREZ_NA_DOBIT_ZATV"] = float("NaN")
        table["POREZ_NA_DOBIT_MAX"] = float("NaN")
        table["POREZ_NA_DOBIT_MIN"] = float("NaN")
        table["INVESTIRANJA_OTV"] = float("NaN")
        table["INVESTIRANJA_ZATV"] = float("NaN")
        table["INVESTIRANJA_MAX"] = float("NaN")
        table["INVESTIRANJA_MIN"] = float("NaN")
        table["OTV-AGR"] = float("NaN")
        table["ZATV-AGR"] = float("NaN")
        table["GDP-OTV"] = float("NaN")
        table["GDP-ZATV"] = float("NaN")
        table["GDP-MAX"] = float("NaN")
        table["GDP-MIN"] = float("NaN")
        table["Export-OTV"] = float("NaN")
        table["Export-ZATV"] = float("NaN")
        table["Export-MAX"] = float("NaN")
        table["Export-MIN"] = float("NaN")
        table["Tax-dorian-OTV"] = float("NaN")
        table["Tax-dorian-ZATV"] = float("NaN")
        table["Tax-dorian-MIN"] = float("NaN")
        table["Tax-dorian-MAX"] = float("NaN")
        table["Import-OTV"] = float("NaN")
        table["Import-ZATV"] = float("NaN")
        table["Import-MAX"] = float("NaN")
        table["Import-MIN"] = float("NaN")
        table["Inflacija-OTV"] = float("NaN")
        table["Inflacija-ZATV"] = float("NaN")
        table["Inflacija-MAX"] = float("NaN")
        table["Inflacija-MIN"] = float("NaN")
        table["ForeignInvest-OTV"]  = float("NaN")
        table["ForeignInvest-ZATV"]  = float("NaN")
        table["ForeignInvest-MAX"]  = float("NaN")
        table["ForeignInvest-MIN"]  = float("NaN")
        table["GrossDSaving-OTV"] = float("NaN")
        table["GrossDSaving-ZATV"] = float("NaN")
        table["GrossDSaving-MAX"] = float("NaN")
        table["GrossDSaving-MIN"] = float("NaN")
        table["GrossSaving-OTV"] = float("NaN")
        table["GrossSaving-ZATV"] = float("NaN")
        table["GrossSaving-MAX"] = float("NaN")
        table["GrossSaving-MIN"] = float("NaN")
        table["GDPPC-OTV"] = float("NaN")
        table["GDPPC-ZATV"] = float("NaN")
        table["GDPPC-MAX"] = float("NaN")
        table["GDPPC-MIN"] = float("NaN")
        table["Turism-OTV"] = float("NaN")
        table["Turism-ZATV"] = float("NaN")
        table["Turism-MAX"] = float("NaN")
        table["Turism-MIN"] = float("NaN")
        table["TarrifLines-OTV"] = float("NaN")
        table["TarrifLines-ZATV"] = float("NaN")
        table["TarrifLines-MAX"] = float("NaN")
        table["TarrifLines-MIN"] = float("NaN")
        table["TaxRevenue-OTV"] = float("NaN")
        table["TaxRevenue-ZATV"] = float("NaN")
        table["TaxRevenue-MAX"] = float("NaN")
        table["TaxOnGoods-MIN"] = float("NaN")
        table["TaxOnGoods-OTV"] = float("NaN")
        table["TaxOnGoods-ZATV"] = float("NaN")
        table["TaxOnGoods-MAX"] = float("NaN")
        table["TaxOnGoods-MIN"] = float("NaN")
        
        row_counter = 0
        num_of_rows = len(self.data.index)
    
        print("Calculating features...")
        for index, row in table.iterrows():
            if index % 1000 == 0:
                print(str(100 * index / len(table)) + "% processed...")
            godina_otvaranja = float("NaN")
            mjesec_otvaranja = "NaN"
            godina_planiranog_zatvaranja = float("NaN")
            mjesec_planiranog_zatvaranja = "NaN"
            dif_otv = float("NaN")
            dif_ztv = float("NaN")
            dif_otv_eu = float("NaN")
            dif_ztv_eu = float("NaN")
            dif_otv_agr = float("NaN")
            dif_ztv_agr = float("NaN")
            god_otv_in_range = False
            god_zatv_in_range = False
            
            #----INICIJALIZACIJA HELPER VARIJABLI
            if row["DATUM_OTVARANJA"] != "NaN":
                datum_otvaranja = row["DATUM_OTVARANJA"]
                godina_otvaranja = MDate.GetYear(datum_otvaranja)
                mjesec_otvaranja = MDate.GetMonthString(datum_otvaranja)
                abs_date = datum_otvaranja + MDate.startDate
                dif_otv = abs_date - self.datum_pocetka_krize
                dif_otv_eu = abs_date - self.datum_ulaska_u_eu
                dif_otv_agr = abs_date - self.datum_pada_agrokora

            if row["PLANIRANI_DATUM_ZATVARANJA"] != "NaN":
                datum_planiranog_zatvaranja = row["PLANIRANI_DATUM_ZATVARANJA"]
                godina_planiranog_zatvaranja = MDate.GetYear(datum_planiranog_zatvaranja)
                mjesec_planiranog_zatvaranja = MDate.GetMonthString(datum_planiranog_zatvaranja)
                abs_date = datum_planiranog_zatvaranja + MDate.startDate
                dif_ztv = abs_date - self.datum_pocetka_krize
                dif_ztv_eu = abs_date - self.datum_ulaska_u_eu
                dif_ztv_agr = abs_date - self.datum_pada_agrokora
            
            #---UBACIVANJE U TABLICU
            if godina_otvaranja < 1995:
                godina_otvaranja = 1995
            if godina_otvaranja > 2017:
                godina_otvaranja = 2017 
            
            if godina_planiranog_zatvaranja < 1995:
                godina_planiranog_zatvaranja = 1995
            if godina_planiranog_zatvaranja > 2017:
                godina_planiranog_zatvaranja = 2017
            
            #OTVARANJA
            
            table.at[index, "RAST_BDPA_U_GODINI_OTVARANJA"] = float(series[0][str(godina_otvaranja)])
            if godina_otvaranja in range(1996,2015):
                table.at[index, "POREZ_NA_DOBIT_OTV"] = float(series[1][str(godina_otvaranja)])
                table.at[index, "INVESTIRANJA_OTV"] = float(series[2][str(godina_otvaranja)])
            table.at[index, "GDP-OTV"] = float(series[4][str(godina_otvaranja)])
            table.at[index, "Export-OTV"] = float(series[5][str(godina_otvaranja)])
            table.at[index, "Tax-dorian-OTV"] = float(series[6][str(godina_otvaranja)])
            table.at[index, "Import-OTV"] = float(series[7][str(godina_otvaranja)])
            table.at[index, "Inflacija-OTV"] = float(series[8][str(godina_otvaranja)])
            table.at[index, "ForeignInvest-OTV"] = float(series[9][str(godina_otvaranja)])
            table.at[index, "GrossDSaving-OTV"] = float(series[10][str(godina_otvaranja)])
            table.at[index, "GrossSaving-OTV"] = float(series[11][str(godina_otvaranja)])
            table.at[index, "GDPPC-OTV"] = float(series[12][str(godina_otvaranja)])
            table.at[index, "Turism-OTV"] = float(series[13][str(godina_otvaranja)])
            table.at[index, "TarrifLines-OTV"] = float(series[14][str(godina_otvaranja)])
            table.at[index, "TaxRevenue-OTV"] = float(series[15][str(godina_otvaranja)])
            table.at[index, "TaxOnGoods-OTV"] = float(series[16][str(godina_otvaranja)])
                    
            #ZATVARANJA 
            
            table.at[index, "RAST_BDPA_U_GODINI_ZATVARANJA"] = float(series[1][str(godina_planiranog_zatvaranja)])
            if godina_planiranog_zatvaranja in range(1996,2015):
                table.at[index, "POREZ_NA_DOBIT_ZATV"] = float(series[2][str(godina_planiranog_zatvaranja)])
                table.at[index, "INVESTIRANJA_ZATV"] = float(series[4][str(godina_planiranog_zatvaranja)])
            table.at[index, "GDP-ZATV"] = float(series[4][str(godina_planiranog_zatvaranja)])
            table.at[index, "Export-ZATV"] = float(series[5][str(godina_planiranog_zatvaranja)])
            table.at[index, "Tax-dorian-ZATV"] = float(series[6][str(godina_planiranog_zatvaranja)])
            table.at[index, "Import-ZATV"] = float(series[7][str(godina_planiranog_zatvaranja)])
            table.at[index, "Inflacija-ZATV"] = float(series[8][str(godina_planiranog_zatvaranja)])
            table.at[index, "ForeignInvest-ZATV"] = float(series[9][str(godina_planiranog_zatvaranja)])
            table.at[index, "GrossDSaving-ZATV"] = float(series[10][str(godina_planiranog_zatvaranja)])
            table.at[index, "GrossSaving-ZATV"] = float(series[11][str(godina_planiranog_zatvaranja)])
            table.at[index, "GDPPC-ZATV"] = float(series[12][str(godina_planiranog_zatvaranja)])
            table.at[index, "Turism-ZATV"] = float(series[13][str(godina_planiranog_zatvaranja)])
            table.at[index, "TarrifLines-ZATV"] = float(series[14][str(godina_planiranog_zatvaranja)])
            table.at[index, "TaxRevenue-ZATV"] = float(series[15][str(godina_planiranog_zatvaranja)])
            table.at[index, "TaxOnGoods-ZATV"] = float(series[16][str(godina_otvaranja)])
            
            #MAX/MIN GENERATION
            maxGDPRast, minGDPRast = Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.GDPRast[0])
            maxTaxOnIncome, minTaxOnIncome = Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.TaxOnIncome)
            maxNonFinacialInvestement, minNonFinacialInvestement = Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.NonFinacialInvestement)
            maxGDP, minGDP =Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.GDP)
            maxExport, minExport = Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.ExportGoods)
            maxTaxDorian, minTaxDorian = Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.TaxesDorian)
            maxImport, minImport =Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.ImportOfGoods)
            maxInflacija, minInflacija =Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.Inflation)
            maxForInvest, minForInvest = Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.ForeignInvest)
            maxGrossDSaving, minGrossDSaving =Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.GrossDomesticSaving)
            maxGrossSaving, minGrossSaving =Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.GrossSaving)
            maxGDPPC, minGDPPC =Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.GDPPC)
            maxTurism, minTurism =Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.Turism)
            maxTarrifLines, minTarrifLines=Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.TariffLines)
            maxTaxRevenue, minTaxRevenue = Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.TaxRevenue)
            maxTaxOnGoods, minTaxOnGoods = Dataset.getMaxAndMin(godina_otvaranja, godina_planiranog_zatvaranja, self.TaxOnGoods)
            
            table.at[index, "MAX_RAST_BDP"] = float(maxGDPRast)
            table.at[index, "MIN_RAST_BDP"] = float(minGDPRast)
                
            table.at[index, "POREZ_NA_DOBIT_MAX"] = float(maxTaxOnIncome)
            table.at[index, "POREZ_NA_DOBIT_MIN"] = float(minTaxOnIncome)
                
            table.at[index, "INVESTIRANJA_MAX"] = float(maxNonFinacialInvestement)
            table.at[index, "INVESTIRANJA_MIN"] = float(minNonFinacialInvestement)
            
            table.at[index, "GDP-MAX"] = float(maxGDP)
            table.at[index, "GDP-MIN"] = float(maxGDP)                                                    
            
            table.at[index, "Export-MAX"] = float(maxExport)
            table.at[index, "Export-MIN"] = float(maxExport)
            
            table.at[index, "Tax-dorian-MAX"] = float(maxTaxDorian)
            table.at[index, "Tax-dorian-MIN"] = float(maxTaxDorian)                                                    
            
            table.at[index, "Import-MAX"] = float(maxImport)
            table.at[index, "Import-MIN"] = float(maxImport)
            
            table.at[index, "Inflacija-MAX"] = float(maxInflacija)
            table.at[index, "Inflacija-MIN"] = float(maxInflacija)                                                    
            
            table.at[index, "ForeignInvest-MAX"] = float(maxForInvest)
            table.at[index, "ForeignInvest-MIN"] = float(maxForInvest)
            
            table.at[index, "GrossDSaving-MAX"] = float(maxGrossDSaving)
            table.at[index, "GrossDSaving-MIN"] = float(minGrossDSaving)
            
            table.at[index, "GrossSaving-MAX"] = float(maxGrossSaving)
            table.at[index, "GrossSaving-MIN"] = float(minGrossSaving)
            
            table.at[index, "GDPPC-MAX"] = float(maxGDPPC)
            table.at[index, "GDPPC-MIN"] = float(minGDPPC)                                                    
            
            table.at[index, "Turism-MAX"] = float(maxTurism)
            table.at[index, "Turism-MIN"] = float(minTurism)
            
            table.at[index, "TarrifLines-MAX"] = float(maxTarrifLines)
            table.at[index, "TarrifLines-MIN"] = float(minTarrifLines)
                                                                
            table.at[index, "TaxRevenue-MAX"] = float(maxTaxRevenue)
            table.at[index, "TaxRevenue-MIN"] = float(minTaxRevenue)
            
            table.at[index, "TaxOnGoods-MAX"] = float(maxTaxOnGoods)
            table.at[index, "TaxOnGoods-MIN"] = float(minTaxOnGoods)
            
            #-----DANI OD KRIZE, ULASKA U EU I PADA AGROKORA
            if dif_otv != float("NaN"):
                table.at[index, "OTV-KRIZA"] = dif_otv
                
            if dif_ztv != float("NaN") :
                table.at[index, "ZATV-KRIZA"] = dif_ztv
                
            if dif_otv_eu != float("NaN"):
                table.at[index, "OTV-EU"] = dif_otv_eu
                
            if dif_ztv_eu != float("NaN") :
                table.at[index, "ZATV-EU"] = dif_ztv_eu
                
            if dif_otv_agr != float("NaN"):
                table.at[index, "OTV-AGR"] = dif_otv_agr
                
            if dif_ztv_agr != float("NaN") :
                table.at[index, "ZATV-AGR"] = dif_ztv_agr
    #------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------------------------------------------------------------------ 
    @staticmethod 
    def getMaxAndMin(startYear, endYear, row):
        currentYear = startYear
        values = []
        while currentYear != endYear + 1:
            try:
                value = float(row[str(currentYear)])
            except KeyError: #dana godina ne postoji u datasetu
                continue
            if value == float("NaN"):
                continue
            #sve radi    
            values.append(value)
            currentYear += 1
        if values: #lista ima elemenata     
            maxval = max(values)
            minval = min(values)
            return maxval, minval
        return float("NaN"), float("NaN")
    
    #------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------------------------------------------------------------------ 
    def getCurrencyMinAndMax(self,dataset):
        minValue = float("NaN")
        maxValue = float("NaN")
        values = dataset["Srednji za devize"].values
        try:
            if len(values):
                minValue = min(values)
                maxValue = max(values)
        except:
            print(values)
                                                            
        return float(minValue.replace(',','.')), float(maxValue.replace(',','.'))
    
    #------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------------------------------------------------------------------ 
    #dodaj nove stupce koji su dobiveni manipuliranjem postojecim stupcima iz filtiriranog data set-a
    def AddPartijaFeatures(self, table):
        table["NUM_PARTIJA"] = 0
        table["NUM_PARTIJA_A"] = 0
        table["NUM_PARTIJA_L"] = 0
        table["RATIO_PARTIJA"] = 0.0
        table["RATIO_PARTIJA_A"] = 0.0
        table["RATIO_PARTIJA_L"] = 0.0
        table["DULJINA_PARTIJE"] = 0
        table["KLIJENT_UKUPNI_IZNOS"] = 0.0
        table["KLIJENT_UKUPNI_IZNOS_A"] = 0.0
        table["KLIJENT_UKUPNI_IZNOS_L"] = 0.0
        clientsA = {}
        clientsL = {}
        clientsAY = {}
        clientsLY = {}
        totalA = {}
        totalL = {}
        for index, row in table.iterrows():
            clientsA[row["KLIJENT_ID"]] = (clientsA.get(row["KLIJENT_ID"]) or 0) + int(row["VRSTA_PROIZVODA"] == "A")
            clientsL[row["KLIJENT_ID"]] = (clientsL.get(row["KLIJENT_ID"]) or 0) + int(row["VRSTA_PROIZVODA"] == "L")
            clientsAY[row["KLIJENT_ID"]] = \
                (clientsAY.get(row["KLIJENT_ID"]) or 0) + \
                int(row["VRSTA_PROIZVODA"] == "A" and row["PRIJEVREMENI_RASKID"] == "Y")
            clientsLY[row["KLIJENT_ID"]] = \
                (clientsLY.get(row["KLIJENT_ID"]) or 0) + \
                int(row["VRSTA_PROIZVODA"] == "L" and row["PRIJEVREMENI_RASKID"] == "Y")
            totalA[row["KLIJENT_ID"]] = \
                (totalA.get(row["KLIJENT_ID"]) or 0) + row["UGOVORENI_IZNOS"] * int(row["VRSTA_PROIZVODA"] == "A")
            totalL[row["KLIJENT_ID"]] = \
                (totalL.get(row["KLIJENT_ID"]) or 0) + row["UGOVORENI_IZNOS"] * int(row["VRSTA_PROIZVODA"] == "L")
        for index, row in table.iterrows():
            table.at[index, "NUM_PARTIJA"] = clientsA[row["KLIJENT_ID"]] + clientsL[row["KLIJENT_ID"]]
            table.at[index, "NUM_PARTIJA_A"] = clientsA[row["KLIJENT_ID"]]
            table.at[index, "NUM_PARTIJA_L"] = clientsL[row["KLIJENT_ID"]]
            table.at[index, "RATIO_PARTIJA"] = \
                (clientsAY[row["KLIJENT_ID"]] + clientsLY[row["KLIJENT_ID"]]) / (clientsA[row["KLIJENT_ID"]] + clientsL[row["KLIJENT_ID"]])
            table.at[index, "RATIO_PARTIJA_A"] = \
                clientsAY[row["KLIJENT_ID"]] / clientsA[row["KLIJENT_ID"]] if clientsA[row["KLIJENT_ID"]] > 0 else None
            table.at[index, "RATIO_PARTIJA_L"] = \
                clientsLY[row["KLIJENT_ID"]] / clientsL[row["KLIJENT_ID"]] if clientsL[row["KLIJENT_ID"]] > 0 else None
            table.at[index, "DULJINA_PARTIJE"] = row["PLANIRANI_DATUM_ZATVARANJA"] - row["DATUM_OTVARANJA"]
            table.at[index, "KLIJENT_UKUPNI_IZNOS"] = totalA[row["KLIJENT_ID"]] + totalL[row["KLIJENT_ID"]]
            table.at[index, "KLIJENT_UKUPNI_IZNOS_A"] = totalA[row["KLIJENT_ID"]]
            table.at[index, "KLIJENT_UKUPNI_IZNOS_L"] = totalL[row["KLIJENT_ID"]]
    
   
    #------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------------------------------------------------------------------ 
    #modificiraj dan test set tako da moze biti predan modelu za ucenje
    def PrepareValidationSet(self):
        #mozgalo test set
        self.testData = pd.read_csv("D:/Mozgalo2019/validation_set_nan.csv", sep = ',', low_memory=False, error_bad_lines = False)
        self.AddAdditionalData(self.testData)
        self.AddPartijaFeatures(self.testData)
        
        trainingClients = {}
        for index, row in self.data.iterrows():
            trainingClients[row["KLIJENT_ID"]] = row

        table = self.testData
        for index, row in table.iterrows():
            if row["KLIJENT_ID"] not in trainingClients:
                table.at[index, "RATIO_PARTIJA"] = float("NaN")
                table.at[index, "RATIO_PARTIJA_A"] = float("NaN")
                table.at[index, "RATIO_PARTIJA_L"] = float("NaN")
                continue
            table.at[index, "NUM_PARTIJA"] += trainingClients[row["KLIJENT_ID"]]["NUM_PARTIJA"]
            table.at[index, "NUM_PARTIJA_A"] += trainingClients[row["KLIJENT_ID"]]["NUM_PARTIJA_A"]
            table.at[index, "NUM_PARTIJA_L"] += trainingClients[row["KLIJENT_ID"]]["NUM_PARTIJA_L"]
            table.at[index, "RATIO_PARTIJA"] = trainingClients[row["KLIJENT_ID"]]["RATIO_PARTIJA"]
            table.at[index, "RATIO_PARTIJA_A"] = trainingClients[row["KLIJENT_ID"]]["RATIO_PARTIJA_A"]
            table.at[index, "RATIO_PARTIJA_L"] = trainingClients[row["KLIJENT_ID"]]["RATIO_PARTIJA_L"]
            table.at[index, "KLIJENT_UKUPNI_IZNOS"] += trainingClients[row["KLIJENT_ID"]]["KLIJENT_UKUPNI_IZNOS"]
            table.at[index, "KLIJENT_UKUPNI_IZNOS_A"] += trainingClients[row["KLIJENT_ID"]]["KLIJENT_UKUPNI_IZNOS_A"]
            table.at[index, "KLIJENT_UKUPNI_IZNOS_L"] += trainingClients[row["KLIJENT_ID"]]["KLIJENT_UKUPNI_IZNOS_L"]
        return trainingClients

In [3]:
#klasa za baratanje sa datumima, svrha je laksa konverzija nasih stringova datume u datetime datum
#atributi klase:
#godina, mjesec i dan - od ucitanog stringa
#date - datum ucitan iz stringa kao datetime objekt
class MDate:    
    startDate = dt.date(1970,1,1).toordinal()  #1.1.1970 je startDate za R
    mjeseci = ["Siječanj", "Veljača", "Ožujak", "Travanj", "Svibanj", "Lipanj", "Srpanj", "Kolovoz", "Rujan", "Listopad", "Studeni", "Prosinac"]
    badAPI = dt.date(1,1,1) #za koristenje form i to ordinal
    
    #konstruktor, prima mozgalo string datuma i izvuce godinu, mjesec i dan, inace baca iznimku
    def __init__(self, string):
        if len(string) != 10 :
            raise Exception("godina nije u formatu dd.mm.gggg") 
        self.dan = int(string[:2])
        self.mjesec = int(string[3:5])
        self.godina = int(string[-4:])
        self.numericDate = dt.date(self.godina, self.mjesec, self.dan).toordinal() 
        
    #metoda za izvalcenje godine u iz numerickog datuma IZ R-a DAKLE OD 1,1,1970 A NE IZ 1,1,1
    @staticmethod    
    def GetYear(num):        
        return MDate.badAPI.fromordinal(num + MDate.startDate).year
    
    @staticmethod    
    def GetMonth(num):
        return MDate.badAPI.fromordinal(num + MDate.startDate).month
    
    @staticmethod    
    def GetMonthString(num):
        return MDate.mjeseci[MDate.badAPI.fromordinal(num + MDate.startDate).month - 1]
    

In [4]:
ds = Dataset()      #inicijalizacija dataseta

In [5]:
ds.LoadAdditionalData()    #loadanje dodatnih podataka

In [6]:
ds.AddAdditionalData(ds.data)     #sporo, ubacivanje podataka u tablicu

Initializing new features...
Calculating features...
0.0% processed...
0.22353661753331813% processed...
0.44707323506663627% processed...
0.6706098525999544% processed...
0.8941464701332725% processed...
1.1176830876665906% processed...
1.3412197051999089% processed...
1.5647563227332268% processed...
1.788292940266545% processed...
2.0118295577998633% processed...
2.2353661753331813% processed...
2.4589027928664993% processed...
2.6824394103998177% processed...
2.9059760279331357% processed...
3.1295126454664537% processed...
3.353049262999772% processed...
3.57658588053309% processed...
3.800122498066408% processed...
4.0236591155997266% processed...
4.2471957331330445% processed...
4.4707323506663625% processed...
4.6942689681996805% processed...
4.9178055857329985% processed...
5.141342203266317% processed...
5.364878820799635% processed...
5.588415438332953% processed...
5.811952055866271% processed...
6.035488673399589% processed...
6.259025290932907% processed...
6.482561908466

56.33122761839617% processed...
56.55476423592949% processed...
56.7783008534628% processed...
57.00183747099612% processed...
57.22537408852944% processed...
57.44891070606276% processed...
57.67244732359608% processed...
57.895983941129394% processed...
58.119520558662714% processed...
58.343057176196034% processed...
58.56659379372935% processed...
58.790130411262666% processed...
59.013667028795986% processed...
59.237203646329306% processed...
59.460740263862625% processed...
59.68427688139594% processed...
59.90781349892926% processed...
60.13135011646258% processed...
60.3548867339959% processed...
60.57842335152922% processed...
60.80195996906253% processed...
61.02549658659585% processed...
61.24903320412917% processed...
61.47256982166249% processed...
61.6961064391958% processed...
61.91964305672912% processed...
62.14317967426244% processed...
62.36671629179576% processed...
62.590252909329074% processed...
62.813789526862394% processed...
63.03732614439571% processed...
63

In [6]:
ds.AddPartijaFeatures(ds.data)

In [7]:
pd.set_option('display.max_columns', 500)

In [8]:
ds.data.sort_values(by=['NUM_PARTIJA'], ascending=True)
ds.data

,Unnamed: 0,Unnamed: 0.1,X,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,DATUM_ZATVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,VISINA_KAMATE,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID,BDP_U_GODINI_OTVARANJA,BDP_U_GODINI_ZATVARANJA,MAX_BDP,MIN_BDP,RAST_BDPA_U_GODINI_OTVARANJA,RAST_BDPA_U_GODINI_ZATVARANJA,MAX_RAST_BDP,MIN_RAST_BDP,OTV-KRIZA,ZATV-KRIZA,OTV-EU,ZATV-EU,NUM_PARTIJA,NUM_PARTIJA_A,NUM_PARTIJA_L,RATIO_PARTIJA,RATIO_PARTIJA_A,RATIO_PARTIJA_L,DULJINA_PARTIJE,KLIJENT_UKUPNI_IZNOS,KLIJENT_UKUPNI_IZNOS_A,KLIJENT_UKUPNI_IZNOS_L
0,0,1,476859,33820.0,163007,11816,15125,15134,1.330964e+06,2,1120,FL1310,A,8.50,C,20.0,N,12279.127217,20758.464152,20758.0,12279.0,5.253364,2.853093,5.581187,-7.177771,-2321.0,988.0,-4071.0,-762.0,4,4,0,0.500000,0.500000,NaN,3309,4.141313e+06,4.141313e+06,0.000000e+00
1,1,2,533133,1618.0,163116,11851,15523,15573,7.341598e+06,2,1120,FL1310,A,0.00,B,20.0,N,12279.127217,21156.644219,21156.0,12279.0,5.253364,-2.001562,5.581187,-7.177771,-2286.0,1386.0,-4036.0,-364.0,165,165,0,0.012121,0.012121,NaN,3672,1.224218e+08,1.224218e+08,0.000000e+00
2,2,3,367799,221009.0,163312,11915,15652,15658,7.483237e+06,2,1120,FL1310,A,0.00,B,23.0,N,12279.127217,21156.644219,21156.0,12279.0,5.253364,-2.001562,5.581187,-7.177771,-2222.0,1515.0,-3972.0,-235.0,4,4,0,0.000000,0.000000,NaN,3737,3.904697e+07,3.904697e+07,0.000000e+00
3,3,4,529164,85962.0,163577,12004,15030,15215,7.933556e+05,2,1430,FL1500,A,6.00,C,15.0,N,12279.127217,20758.464152,20758.0,12279.0,5.253364,2.853093,5.581187,-7.177771,-2133.0,893.0,-3883.0,-857.0,1,1,0,0.000000,0.000000,NaN,3026,7.933556e+05,7.933556e+05,0.000000e+00
4,4,5,306996,1061.0,163718,12038,15701,15397,1.411624e+06,2,1120,FL1500,A,6.80,C,16.0,Y,12279.127217,21156.644219,21156.0,12279.0,5.253364,-2.001562,5.581187,-7.177771,-2099.0,1564.0,-3849.0,-186.0,3,3,0,0.666667,0.666667,NaN,3663,4.865950e+06,4.865950e+06,0.000000e+00
5,5,6,242812,511687.0,163825,12079,15730,15736,1.127470e+06,2,1430,FL1310,A,10.00,C,17.0,N,13126.009572,21807.107966,21807.0,13126.0,5.581187,-0.214223,5.581187,-7.177771,-2058.0,1593.0,-3808.0,-157.0,1,1,0,0.000000,0.000000,NaN,3651,1.127470e+06,1.127470e+06,0.000000e+00
6,6,7,546910,33820.0,164109,12157,15784,15805,8.354421e+05,2,1120,FL1310,A,8.50,C,22.0,N,13126.009572,21807.107966,21807.0,13126.0,5.581187,-0.214223,5.581187,-7.177771,-1980.0,1647.0,-3730.0,-103.0,4,4,0,0.500000,0.500000,NaN,3627,4.141313e+06,4.141313e+06,0.000000e+00
7,7,8,538176,1196781.0,164426,12256,15876,14986,2.850111e+07,2,1110,FL1400,A,0.00,B,2.0,Y,13126.009572,21807.107966,21807.0,13126.0,5.581187,-0.214223,5.581187,-7.177771,-1881.0,1739.0,-3631.0,-11.0,2,2,0,1.000000,1.000000,NaN,3620,4.249076e+07,4.249076e+07,0.000000e+00
8,8,9,296033,4324.0,164527,12292,15891,15908,7.442962e+06,2,1420,FE0100,A,0.00,B,33.0,N,13126.009572,21807.107966,21807.0,13126.0,5.581187,-0.214223,5.581187,-7.177771,-1845.0,1754.0,-3595.0,4.0,3,3,0,0.000000,0.000000,NaN,3599,8.153962e+06,8.153962e+06,0.000000e+00
9,9,10,140940,195391.0,164971,12408,16405,16416,1.090687e+07,2,1110,FL1500,A,4.00,A,14.0,N,13126.009572,22077.225626,22077.0,13126.0,5.581187,0.320303,5.581187,-7.177771,-1729.0,2268.0,-3479.0,518.0,10,9,1,0.200000,0.222222,0.000000,3997,7.080855e+07,6.896507e+07,1.843486e+06


In [11]:
ds.data.to_csv("D:/Mozgalo2019/training_set_economy_features_extended.csv")

In [9]:
trainingClients = ds.PrepareValidationSet()

Initializing new features...
Calculating features...
0.0% processed...
1.1778146826378337% processed...
2.3556293652756675% processed...
3.5334440479135014% processed...
4.711258730551335% processed...
5.889073413189169% processed...
7.066888095827003% processed...
8.244702778464836% processed...
9.42251746110267% processed...
10.600332143740504% processed...
11.778146826378338% processed...
12.955961509016172% processed...
14.133776191654006% processed...
15.31159087429184% processed...
16.489405556929672% processed...
17.667220239567506% processed...
18.84503492220534% processed...
20.022849604843174% processed...
21.200664287481008% processed...
22.37847897011884% processed...
23.556293652756676% processed...
24.73410833539451% processed...
25.911923018032343% processed...
27.089737700670177% processed...
28.26755238330801% processed...
29.445367065945845% processed...
30.62318174858368% processed...
31.800996431221513% processed...
32.978811113859344% processed...
34.15662579649718

In [10]:
ds.testData

,Unnamed: 0,X,instance_id,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,VISINA_KAMATE,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID,RAST_BDPA_U_GODINI_OTVARANJA,RAST_BDPA_U_GODINI_ZATVARANJA,MAX_RAST_BDP,MIN_RAST_BDP,OTV-KRIZA,ZATV-KRIZA,OTV-EU,ZATV-EU,OTV_TECAJ,ZATV_TECAJ,MAX_TECAJ,MIN_TECAJ,POREZ_NA_DOBIT_OTV,POREZ_NA_DOBIT_ZATV,POREZ_NA_DOBIT_MAX,POREZ_NA_DOBIT_MIN,INVESTIRANJA_OTV,INVESTIRANJA_ZATV,INVESTIRANJA_MAX,INVESTIRANJA_MIN,OTV-AGR,ZATV-AGR,GDP-OTV,GDP-ZATV,GDP-MAX,GDP-MIN,Export-OTV,Export-ZATV,Export-MAX,Export-MIN,Tax-dorian-OTV,Tax-dorian-ZATV,Tax-dorian-MIN,Tax-dorian-MAX,Import-OTV,Import-ZATV,Import-MAX,Import-MIN,Inflacija-OTV,Inflacija-ZATV,Inflacija-MAX,Inflacija-MIN,ForeignInvest-OTV,ForeignInvest-ZATV,ForeignInvest-MAX,ForeignInvest-MIN,GrossDSaving-OTV,GrossDSaving-ZATV,GrossDSaving-MAX,GrossDSaving-MIN,GrossSaving-OTV,GrossSaving-ZATV,GrossSaving-MAX,GrossSaving-MIN,GDPPC-OTV,GDPPC-ZATV,GDPPC-MAX,GDPPC-MIN,Turism-OTV,Turism-ZATV,Turism-MAX,Turism-MIN,TarrifLines-OTV,TarrifLines-ZATV,TarrifLines-MAX,TarrifLines-MIN,TaxRevenue-OTV,TaxRevenue-ZATV,TaxRevenue-MAX,TaxOnGoods-MIN,TaxOnGoods-OTV,TaxOnGoods-ZATV,TaxOnGoods-MAX,TaxRevenue-MIN,NUM_PARTIJA,NUM_PARTIJA_A,NUM_PARTIJA_L,RATIO_PARTIJA,RATIO_PARTIJA_A,RATIO_PARTIJA_L,DULJINA_PARTIJE,KLIJENT_UKUPNI_IZNOS,KLIJENT_UKUPNI_IZNOS_A,KLIJENT_UKUPNI_IZNOS_L
0,1,1,0,1049000,5585453,15535,15901,273555.01,2,1410,TM0109,L,3.60,A,64.0,NaN,-2.001562,12.274006,-0.214223,-2.001562,1398.0,1764.0,-352.0,14.0,NaN,NaN,NaN,NaN,13.860453,1.101197,13.860453,12.274006,0.947046,0.027504,1.101197,0.947046,-1633.0,-1267.0,-0.103553,0.027504,0.027504,0.027504,0.021851,0.031231,0.031231,0.031231,0.065291,0.041073,0.065291,0.065291,-0.000736,0.031835,0.031835,0.031835,-0.070642,-0.968803,-0.070642,-0.070642,-0.590357,0.453331,0.453331,0.453331,-0.020424,-0.002147,-0.002147,-0.020424,-0.035893,0.049529,0.049529,-0.035893,-0.020424,-0.002147,-0.002147,-0.020424,0.042627,0.052886,0.052886,0.042627,-0.004276,0.035106,0.035106,-0.004276,0.025888,0.029901,0.029901,0.012309,0.012309,0.012309,0.050025,0.025888,2,0,2,NaN,NaN,NaN,366,3.938000e+05,0.000000e+00,3.938000e+05
1,2,2,1,481302,752335,16824,16824,50970.78,2,1410,TM0109,L,0.20,A,56.0,NaN,4.261740,NaN,4.261740,4.261740,2687.0,2687.0,937.0,937.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-344.0,-344.0,0.041648,0.041648,0.041648,0.041648,0.045356,0.045356,0.045356,0.045356,0.038002,0.038002,0.038002,0.038002,0.037273,0.037273,0.037273,0.037273,1.325666,1.325666,1.325666,1.325666,0.941321,0.941321,0.941321,0.941321,0.040875,0.040875,0.040875,0.040875,-0.054940,-0.054940,-0.054940,-0.054940,0.040875,0.040875,0.040875,0.040875,0.081541,0.081541,0.081541,0.081541,-0.038534,-0.038534,-0.038534,-0.038534,0.027375,0.027375,0.027375,-0.027807,-0.027807,-0.027807,-0.027807,0.027375,4,0,4,NaN,NaN,NaN,0,1.993543e+05,0.000000e+00,1.993543e+05
2,3,3,2,838192,6623548,17891,17891,8969.36,2,1410,TM0109,L,0.02,A,52.0,NaN,4.134981,NaN,4.134981,4.134981,3754.0,3754.0,2004.0,2004.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,723.0,723.0,0.064699,0.064699,0.064699,0.064699,0.083237,0.083237,0.083237,0.083237,0.063910,0.063910,0.063910,0.063910,0.095859,0.095859,0.095859,0.095859,1.070539,1.070539,1.070539,1.070539,1.444691,1.444691,1.444691,1.444691,0.039980,0.039980,0.039980,0.039980,0.069244,0.069244,0.069244,0.069244,0.039980,0.039980,0.039980,0.039980,0.114410,0.114410,0.114410,0.114410,-0.030308,-0.030308,-0.030308,-0.030308,0.009845,0.009845,0.009845,0.008031,0.008031,0.008031,0.008031,0.009845,1,0,1,NaN,NaN,NaN,0,8.969360e+03,0.000000e+00,8.969360e+03
3,4,4,3,174575,425060,15556,15556,16142.48,2,1410,TM0109,L,2.30,C,76.0,NaN,-2.001562,13.860453,-2.001562,-2.001562,1419.0,1419.0,-331.0,-331.0,NaN,NaN,NaN,NaN,13.860453,0.947046,13.860453,13.860453,0.947046,-0.103553,0.947046,0.947046,-1612.0,-1612.0,-0.103553,-0.103553,-0.103553,-0.103553,0.021851,0.021851,0.021851

In [11]:
ds.testData.to_csv("D:/Mozgalo2019/validation_set_prepared_extended_merge.csv")